<a href="https://colab.research.google.com/github/GonMazzini/Loads_Surrogate_Transferability/blob/main/PyTorch_Model_v4_Random_HlHuLr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GridSearch:

-This notebook uses a 3 point GridSearch for learning rate (0.1,0.01,0.001) and number of hidden units (16,256,512).
-The objective was set to be the validation error (MSE).
-The best result was obtained for 512 units and 0.01 lr.

In [1]:
!pip install parameter-sherpa

     |████████████████████████████████| 513 kB 10.7 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 959 kB 30.6 MB/s 
     |████████████████████████████████| 71 kB 6.8 MB/s 
  Created wheel for parameter-sherpa: filename=parameter_sherpa-1.0.6-py2.py3-none-any.whl size=542134 sha256=fc59a8dd6754c7c7e944d6c06ef807473027d6f8e0f25877c1710cb9bce277ae
  Stored in directory: /root/.cache/pip/wheels/96/d9/cb/99569566e5e9b3ef0265ba4cbce3ff16f7692988833aa942f5
  Created wheel for GPyOpt: filename=GPyOpt-1.2.6-py3-none-any.whl size=83609 sha256=9edd2e33ddc259db6220eb4248bfda4113f3aba3edbd76887f9a71ed5563a908
  Stored in directory: /root/.cache/pip/wheels/e6/fa/d1/f9652b5af79f769a0ab74dbead7c7aea9a93c6bc74543fd3ec
  Created wheel for GPy: filename=GPy-1.10.0-cp37-cp37m-linux_x86_64.whl size=2565102 sha256=a6afd1fd850096c663c295aae8d90afdb389e4a0503b124ffc7a04a12f4c4fc5
  Stored in directory: /root/.cache/pip/wheels/f7/18/28/dd1ce0192a81b71a3b0

In [2]:
from __future__ import print_function
import sherpa
from sherpa.algorithms import Genetic
import time

In [3]:
import pandas as pd 
import numpy as np
import math
from random import shuffle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU")
else:
    device = torch.device("cpu")
    print("CPU")

GPU


# 0- Read the data as a data frame

In [6]:
df = pd.read_excel('LoadsDataBase_6D_Set123_FiltMinMaxCrit.xlsx') # Average the values from Set1,Set2 and Set3.
df.head(2)
# 0 : TT_Mx_avg
# 1 : TT_My_avg
# 2 : TB_Mx_avg
# 3 : TB_My_avg
# 4 : MS_Mz_avg
# 5 : BR_Mx_avg
# 6 : BR_My_avg
# 7 : TT-Mz_avg

,Unnamed: 0,pointno,U,SigmaU,Alpha,MannL,MannGamma,VeerDeltaPhi,TT_Mx_avg,TT_My_avg,TB_Mx_avg,TB_My_avg,TT_Mz_avg,MS_Mz_avg,BR_Mx_avg,BR_My_avg
0,0,1,4.000000,0.100000,-0.650000,7.500000,1.000000,-22.250000,747.561872,200.666288,6708.717789,8861.885588,819.209904,63.457528,4253.317748,15006.726860
1,1,2,10.150758,1.208656,-0.139692,48.470634,1.363636,-4.771217,3556.031457,676.339081,16692.647572,6329.099515,3746.460605,1354.995442,10409.290476,16289.414152


# ============================================================
# ==========================Section 1 ==========================
# ============================================================

In [7]:
X = df.iloc[:,2:8]
y = df.iloc[:,8:].drop(['TT_Mx_avg','TT_Mz_avg'], axis = 1)

In [8]:
# Test split:
X, X_test, y, y_test = train_test_split(X,y, test_size = 0.2, shuffle = True,  random_state = 101)

print(f'The filtered data set consits on: {len(df)} entries.')
print(f'A total of {len(X)} will be used for training and validation.')
print(f'A total of {len(X_test)} will be used for testing the final model.')


The filtered data set consits on: 7664 entries.
A total of 6131 will be used for training and validation.
A total of 1533 will be used for testing the final model.


### From now on, "X" and "y" will be used for train-validate the model. 

In [9]:
feature_range = (0, 1)
scaler_x = preprocessing.MinMaxScaler(feature_range=feature_range).fit(X)
X_scaled = scaler_x.transform(X)

### Separte between train and validation

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled,y.values, test_size = 0.2, shuffle = True,  random_state = 101)

In [11]:
# printing number of samples for train-validation-test
print(f'A total of {y_train.shape[0]} for training, {round(100*y_train.shape[0]/len(df),1)} % of total data')
print(f'A total of {y_val.shape[0]} for validation, {round(100*y_val.shape[0]/len(df),1)} % of total data')
print(f'A total of {y_test.shape[0]} for testing, {round(100*y_test.shape[0]/len(df),1)} % of total data')

A total of 4904 for training, 64.0 % of total data
A total of 1227 for validation, 16.0 % of total data
A total of 1533 for testing, 20.0 % of total data


### The PyTorch worfklow can be summarized as follows:
- 1) Design model (input, output size, forward pass)
- 2) Construct loss and optimizer
- 3) Training loop
   - forward pass: compute prediction based on the current weights and biases of the net
   - backward pass: compute the gradients of the loss function wrt. to model parameters
   - update weigths in

In [13]:
input_size = np.shape(X_train)[1]             # np.shape(X_train)[1]
output_channels = np.shape(y_train)[1]        # np.shape(y_train)[1]
#hidden_size = 50

In [14]:
class MLP(nn.Module):
    """ A feedforward network designed for tuning number of layers and hidden units.
    By @GonMazzini"""
    def __init__(self, input_dim, output_dim, n_hidLayers, hidden_size):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_size = hidden_size
        self.n_hidLayers = n_hidLayers
        current_dim = input_dim
        self.layers = nn.ModuleList()
        
        for hdim in [self.hidden_size]*self.n_hidLayers:
            self.layers.append(nn.Linear(current_dim, hdim))
            current_dim = hdim
        self.layers.append(nn.Linear(current_dim, output_dim))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        out = F.relu(self.layers[-1](x))
        return out 

In [15]:
# test dummy forward
testMLP = MLP(input_size,output_channels,4,50) 
testMLP(torch.tensor(X_train[0]).float())

tensor([0.0000, 0.0441, 0.0623, 0.0593, 0.0000, 0.1384],
       grad_fn=<ReluBackward0>)

In [16]:
# instantiate the class
#model = Net(hidden_size)  # will be instantiated inside HP loop

# Define the loss function (mean square error)
loss = nn.MSELoss()

# Instantiate optimizer passing the net parameters as argument, and learning rate
#optimizer = optim.Adam(model.parameters(), lr = 0.01)  # will be instantiated inside HP loop 

# list to store results
train_losses , val_losses= [],[]

# Try a dummy forward
#model(torch.tensor(X_train[0]).float())  # beware that need to convert from double to float

# 2.1- Use the PyTorch DataLoader and Dataset utils.
- DataLoader class combines a dataset and a sampler, and provides an iterable over the given dataset for training the model
- Dataset: just an abstract class representing a :class:`Dataset`

In [17]:
class FatigueLoads_TrainSet(Dataset):

    def __init__(self):
        self.n_samples = X_train.shape[0]
        self.x_data = torch.from_numpy(X_train) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(y_train) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
class FatigueLoads_ValidationSet(Dataset):

    def __init__(self):
        self.n_samples = X_val.shape[0]
        self.x_data = torch.from_numpy(X_val) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(y_val) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

### Get first sample and unpack. Note that the enviromental inputs are normalized using MinMaxScaler

In [18]:
train_dataset = FatigueLoads_TrainSet()
valid_dataset = FatigueLoads_ValidationSet()

first_data = train_dataset[0]
features, loads = first_data
print(features, loads)

tensor([0.9213, 0.5395, 0.2971, 0.1574, 0.7369, 0.3075], dtype=torch.float64) tensor([  808.7441, 21966.9989, 16383.7768,   807.4632, 18567.6736, 16935.8620],
       dtype=torch.float64)


In [22]:
batch_size = 64
num_epochs = 1000

num_batches_train = X_train.shape[0] // batch_size
num_batches_test = X_val.shape[0] // batch_size
print(f'batches train: {num_batches_train}')
print(f'batches test:  {num_batches_test}')

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

batches train: 76
batches test:  19


In [20]:
from itertools import repeat
#algorithm = sherpa.algorithms.GridSearch(num_grid_points=3)
algorithm = sherpa.algorithms.RandomSearch(max_num_trials = 32)
# algorithm = sherpa.algorithms.RandomSearch(max_num_trials=15)

# parameters = [sherpa.Discrete('hidden_size', [16,512]),
#               sherpa.Continuous('lr',[0.001,0.1], scale='log')]

parameters = [sherpa.Discrete('n_hidLayers', [2, 6]),
              sherpa.Discrete('hidden_size', [16, 64]),
              sherpa.Continuous('lr',[0.1,0.001],'log')]

In [21]:
study = sherpa.Study(parameters=parameters,
                     algorithm=algorithm,
                     lower_is_better=True)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://172.28.0.2:8880 if on a cluster or
http://localhost:8880 if running locally.
-------------------------------------------------------


 * Serving Flask app "sherpa.app.app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [23]:
for trial in study:
  print("Trial {}:\t{}".format(trial.id, trial.parameters))
  # model = Net(trial.parameters['hidden_size'])
  model=MLP(input_dim=np.shape(X_train)[1] , 
            output_dim=np.shape(y_train)[1] ,
            n_hidLayers=trial.parameters['n_hidLayers'],
            hidden_size=trial.parameters['hidden_size'],
            )    
  #model.train()
  model.to(device)
  optimizer = optim.Adam(model.parameters(), lr = trial.parameters['lr']) # lr = trial.parameters['lr']
  
  for epoch in range(num_epochs):

    epoch_loss = 0
    model.train()
    
    for i, (inputs, loads) in enumerate(train_loader):
        #print(features, loads)
        
        optimizer.zero_grad()                      # zeroize accumulated gradients in parameters             
        
        output = model(inputs.float().to(device))             # forwards pass       
        batch_loss = loss(output, loads.float().to(device))   # compute loss for current batch
        
        batch_loss.backward()                      # compute the gradient of the loss wrt. model parameters
        optimizer.step()                           # update weights according to the comptued gradients
        
    
    epoch_loss_train = 0
    epoch_loss_test = 0
    model.eval()
    
    ##### Evaluate training
    for i, (inputs, loads) in enumerate(train_loader):
        
        output = model(inputs.float().to(device))
        
        batch_loss_train = loss(output, loads.float().to(device))  # compute loss for the current batch
        epoch_loss_train += batch_loss_train            # accumulate loss for the current epoch
        
        #print(f'Epoch: {epoch+1}/{num_epochs}  | Step {i+1}/{n_iterations}')
    
    ##### Evaluate validation    
    for i, (inputs, loads) in enumerate(valid_loader):
        
        output = model(inputs.float().to(device))
        
        batch_loss_test = loss(output, loads.float().to(device))  # compute loss for the current batch
        epoch_loss_test += batch_loss_test     # accumulate loss for the current epoch
        
        #print(f'Epoch: {epoch+1}/{num_epochs}  | Step {i+1}/{n_iterations}')
    
    if epoch % 20 == 0: 
        print(f'Epoch: {epoch+1}/{num_epochs} | Train loss: {0.001*epoch_loss_train/num_batches_train:.0f}       | Val loss {0.001*epoch_loss_test/num_batches_test:.0f}')

        study.add_observation(trial=trial,
                              iteration=epoch,
                              objective=0.001*epoch_loss_test.cpu().detach().numpy()/num_batches_test)
        
    if study.should_trial_stop(trial):
        break 
    # store in list for plotting the loss per epoch    
    val_losses.append(0.001*epoch_loss_test.cpu().detach().numpy()/num_batches_test)  
    train_losses.append(0.001*epoch_loss_train.cpu().detach().numpy()/num_batches_train)  

  study.finalize(trial) 

  #study.get_best_result()

Trial 1:	{'n_hidLayers': 5, 'hidden_size': 47, 'lr': 0.0018399878553151303}
Epoch: 1/1000 | Train loss: 157242       | Val loss 166274
Epoch: 21/1000 | Train loss: 17047       | Val loss 17233
Epoch: 41/1000 | Train loss: 16438       | Val loss 16637
Epoch: 61/1000 | Train loss: 16332       | Val loss 16622
Epoch: 81/1000 | Train loss: 16414       | Val loss 16650
Epoch: 101/1000 | Train loss: 16281       | Val loss 16687
Epoch: 121/1000 | Train loss: 16204       | Val loss 16586
Epoch: 141/1000 | Train loss: 16126       | Val loss 16680
Epoch: 161/1000 | Train loss: 16037       | Val loss 16874
Epoch: 181/1000 | Train loss: 6790       | Val loss 7023
Epoch: 201/1000 | Train loss: 3109       | Val loss 3267
Epoch: 221/1000 | Train loss: 2682       | Val loss 2863
Epoch: 241/1000 | Train loss: 2627       | Val loss 2804
Epoch: 261/1000 | Train loss: 2485       | Val loss 2633
Epoch: 281/1000 | Train loss: 2435       | Val loss 2666
Epoch: 301/1000 | Train loss: 2324       | Val loss 248

In [27]:
df = study.results

In [28]:
from google.colab import files
df.to_csv('filename.csv') 
files.download('filename.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
study.get_best_result()

{'Iteration': 440,
 'Objective': 566.0530526315789,
 'Trial-ID': 15,
 'hidden_size': 53,
 'lr': 0.04078152864919068,
 'n_hidLayers': 5}

In [ ]:
study.get_best_result()

In [ ]:
study.results.Objective.describe()

In [ ]:
study.get_best_result()

In [ ]:
study.results

# 3.1- Start training loop 

In [ ]:
for epoch in range(num_epochs):
    
    epoch_loss = 0
    model.train()
    
    for i, (inputs, loads) in enumerate(train_loader):
        #print(features, loads)
        
        optimizer.zero_grad()                      # zeroize accumulated gradients in parameters             
        
        output = model(inputs.float())             # forwards pass       
        batch_loss = loss(output, loads.float())   # compute loss for current batch
        
        batch_loss.backward()                      # compute the gradient of the loss wrt. model parameters
        optimizer.step()                           # update weights according to the comptued gradients
        
    
    epoch_loss_train = 0
    epoch_loss_test = 0
    model.eval()
    
    ##### Evaluate training
    for i, (inputs, loads) in enumerate(train_loader):
        
        output = model(inputs.float())
        
        batch_loss_train = loss(output, loads.float())  # compute loss for the current batch
        epoch_loss_train += batch_loss_train            # accumulate loss for the current epoch
        
        #print(f'Epoch: {epoch+1}/{num_epochs}  | Step {i+1}/{n_iterations}')
    
    ##### Evaluate validation    
    for i, (inputs, loads) in enumerate(valid_loader):
        
        output = model(inputs.float())
        
        batch_loss_test = loss(output, loads.float())  # compute loss for the current batch
        epoch_loss_test += batch_loss_test     # accumulate loss for the current epoch
        
        #print(f'Epoch: {epoch+1}/{num_epochs}  | Step {i+1}/{n_iterations}')
    
    if epoch % 100 == 0: 
        print(f'Epoch: {epoch+1}/{num_epochs} | Train loss: {epoch_loss_train/num_batches_train}       | Val loss {epoch_loss_test/num_batches_test}')
        
    # store in list for plotting the loss per epoch    
    val_losses.append(epoch_loss_test/num_batches_test)  
    train_losses.append(epoch_loss_train/num_batches_train)  

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(train_losses[50:])
plt.plot(val_losses[50:], '-k')
plt.title('Training-Validation loss', fontsize = 16)
plt.grid()
plt.legend(['Training loss','Validation loss'])

# Evalute on the test data

### Scale the test data before evaluating the model

In [ ]:
Test_scaler_x = preprocessing.MinMaxScaler(feature_range=feature_range).fit(X_test) # maybe try another for X?
X_Test_scaled = Test_scaler_x.transform(X_test)

predictions = model(torch.tensor(X_Test_scaled).float()).detach().numpy()
test_targets = y_test.values

In [ ]:
for idx, ch in enumerate(df.columns.tolist()[8:]):
  # issues with 2 and 7
    plt.figure(idx, figsize = (8,6))
    sns.scatterplot(predictions[:,idx],test_targets[:,idx])
    plt.title(f'DELs: {ch}', fontsize = 20)
    plt.xlabel('Predictions [kNm]', fontsize = 12)
    plt.ylabel('Targets [kNm]')

In [ ]:
mse_list = []
r2_score_list = []
for i in range(len(df.columns.tolist()[8:])):
    #print(f'MSE {AllTargetData.columns[i]} Channel : \n {mean_squared_error(AllTargetData.values[:,i], Yout[:,i])}')
    mse_list.append(mean_squared_error(test_targets[:,i], predictions[:,i]))
    r2_score_list.append(r2_score(test_targets[:,i], predictions[:,i]))
 

 # Compute the normalized mean square error:
Norm_RMSE = np.sqrt(np.array(mse_list)) / y_test.describe().loc['mean'].values

In [ ]:
fig = plt.figure(figsize= (14,4)) 
sns.barplot(x= df.columns.tolist()[8:], y= r2_score_list) 
plt.ylim([min(r2_score_list)*0.92,max(r2_score_list)*1.02])
plt.ylabel('R2 score', fontsize = 20)
plt.grid()

In [ ]:
fig = plt.figure(figsize= (10,4)) 
sns.barplot(x= df.columns.tolist()[8:], y= Norm_RMSE) 
plt.ylim([0,max(Norm_RMSE)*1.02])
plt.ylabel('Normalized Root Mean Square Error', fontsize = 20)
plt.grid()

# ============================================================
# ==========================Section 2 ==========================
# ============================================================
### Compare versus the wind2loads neural net

In [ ]:
error on purpose

In [ ]:
from w2l import neuralnets

In [ ]:
# define the model using the same net architecture, and train for the same number of epochs using the previously batch size as well.
w2l_net = neuralnets.ann(layersizes = [input_size, num_hid_1, num_hid_2, output_channels],
                       params = {'minibatchsize':64, 'nepochs':500}, 
                       output_style = 'None',
                        testratios = [0.7, 0.3, 0.])

In [ ]:
# train using the data from the first split
Outdata = w2l_net.train(X.values,y.values)

In [ ]:
w2l_net.params

In [ ]:
Outdata.keys()

$\color{red}{\text{Why 500 epochs tho}}$

In [ ]:
plt.plot(Outdata['Jhist'][50:])


In [ ]:
Yout = w2l_net.predict(X_test.values) 

In [ ]:
for idx, ch in enumerate(df.columns.tolist()[8:]):
  # issues with 2 and 7
    plt.figure(idx, figsize = (8,6))
    sns.scatterplot(Yout[:,idx],test_targets[:,idx])
    sns.scatterplot(predictions[:,idx],test_targets[:,idx])
    plt.title(f'DELs: {ch}', fontsize = 20)
    plt.legend(['W2L','PyTorch'])
    plt.xlabel('Predictions [kNm]', fontsize = 12)
    plt.ylabel('Targets [kNm]')

In [ ]:
W2L_mse_list = []
W2L_r2_score_list = []
for i in range(len(df.columns.tolist()[8:])):
    #print(f'MSE {AllTargetData.columns[i]} Channel : \n {mean_squared_error(AllTargetData.values[:,i], Yout[:,i])}')
    W2L_mse_list.append(mean_squared_error(test_targets[:,i], Yout[:,i]))
    W2L_r2_score_list.append(r2_score(test_targets[:,i], Yout[:,i]))
    
W2L_Norm_RMSE = np.sqrt(np.array(W2L_mse_list)) / y_test.describe().loc['mean'].values

In [ ]:
fig = plt.figure(figsize= (12,6)) 
sns.barplot(x= df.columns.tolist()[8:], y= W2L_r2_score_list) 
plt.ylim([min(W2L_r2_score_list)*0.92,max(W2L_r2_score_list)*1.02])
plt.ylabel('R2 score', fontsize = 20)
plt.grid()

# Compare Wind2Loads net versus PyTorch implementation

In [ ]:
barplot_lst = []
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst.append(['pytorch',ch,r2_score_list[i]])
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst.append(['W2L',ch,W2L_r2_score_list[i]])  

In [ ]:
df_comparison = pd.DataFrame(barplot_lst,
                  columns=['Model','channel','r2'])

In [ ]:
plt.figure(figsize=(12,6))
df_comparison.pivot("channel", "Model", "r2").plot(kind='bar')
plt.ylim([min(r2_score_list)*0.98,max(r2_score_list)*1.02])
plt.title('R2', fontsize = 18)
plt.show()

In [ ]:
barplot_lst_NMSE = []
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst_NMSE.append(['pytorch',ch , Norm_RMSE[i]])
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst_NMSE.append(['W2L', ch, W2L_Norm_RMSE[i]])

In [ ]:
df_NRMSE_comparison = pd.DataFrame(barplot_lst_NMSE,
                  columns=['Model','channel','NRMSE'])



In [ ]:
plt.figure(figsize=(14,6))
df_NRMSE_comparison.pivot("channel", "Model", "NRMSE").plot(kind='bar')
plt.ylim([0,max(W2L_Norm_RMSE)*1.02])
plt.show()